# Downloading stamps (single object example)
```Author: Eden Girma, Last updated 20210503```

# Table of contents:
* [Specify ZTF object ID and detections](#oid)
* [Construct methods to extract stamps data](#methods)

**Goal:**

Given a classified ZTF object: to download stamp images as ```.fits``` files

These objects will have been filtered by another processes, to see if they are actually in the Chandra footprint.

In [1]:
import sys
import os #necessary for making new directory

# Packages for direct database access
# %pip install psycopg2
import psycopg2
import json

# Packages for data and number handling
import numpy as np
import pandas as pd
import math

# Packages for calculating current time and extracting ZTF data to VOTable
from astropy.time import Time
from astropy.table import Table
from astropy.io.votable import from_table, writeto
import astropy.units as units
from astropy.coordinates import SkyCoord
from datetime import datetime

# Handling FITS files
from astropy.io import fits

# Packages for display and data plotting, if desired
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import ephem
import copy
import pickle

In [2]:
from alerce.core import Alerce
client = Alerce()

## Specify ZTF object ID and detections<a class="anchor" id="oid"></a>

In [3]:
obj_oid = 'ZTF19acymzwg'

detections = client.query_detections(oid=obj_oid, format='pandas')
display(detections)

# Print detections dataframe columns:
detections.columns.tolist()

mjd               candid  fid            pid  diffmaglim  \
0  58831.569803  1077569802115015001    1  1077569802115   18.976100   
1  58846.553056  1092553052115015022    1  1092553052115   20.731081   
2  58846.572083  1092572082115015003    2  1092572082115   20.162014   
3  58850.516053  1096516052115015012    2  1096516052115   19.971603   
4  58850.548935  1096548932115015015    1  1096548932115   20.483500   
5  58853.527442  1099527442115015024    1  1099527442115   20.760468   
6  58853.554919  1099554912115015025    2  1099554912115   20.647800   
7  58856.557639  1102557632115015020    1  1102557632115   20.644367   

   isdiffpos   nid    distnr     magpsf  magpsf_corr  ... magapbig  \
0          1  1077  0.639813  19.345400    18.682121  ...  19.4852   
1          1  1092  1.326210  19.916887    18.954903  ...  19.4418   
2          1  1092  0.620091  19.904514    18.489037  ...  19.7133   
3          1  1096  1.228304  20.186290    18.558607  ...  19.5375   
4          1  1096  1.648970  19.995500          NaN  ...  19.4823   
5          1  1099  1.350408  20.155785    19.047123  ...  19.7932   
6          1  1099  0.911095  20.294432    18.581839  ...  20.0368   
7          1  1102  1.591064  20.203712          NaN  ...  19.6838   

   sigmagapbig         rfid  has_stamp  corrected  dubious  candid_alert  \
0       0.5012          NaN      False       True    False          None   
1       0.1427  719120121.0      False       True    False          None   
2       0.3220  719120221.0      False       True    False          None   
3       0.2454  719120221.0      False       True    False          None   
4       0.1532          NaN      False      False     True          None   
5       0.1969  719120121.0      False       True    False          None   
6       0.2988  719120221.0      False       True    False          None   
7       0.1845  719120121.0       True      False     True          None   

  step_id_corr  phase        parent_candid  
0   bulk_1.0.0   None  1099554912115015025  
1   bulk_1.0.0   None  1099554912115015025  
2   bulk_1.0.0   None  1099554912115015025  
3   bulk_1.0.0   None  1099554912115015025  
4   bulk_1.0.0   None  1099554912115015025  
5   bulk_1.0.0   None  1099554912115015025  
6   bulk_1.0.0   None  1102557632115015020  
7   bulk_1.0.0   None                    0  

[8 rows x 33 columns]

['mjd',
 'candid',
 'fid',
 'pid',
 'diffmaglim',
 'isdiffpos',
 'nid',
 'distnr',
 'magpsf',
 'magpsf_corr',
 'magpsf_corr_ext',
 'magap',
 'magap_corr',
 'sigmapsf',
 'sigmapsf_corr',
 'sigmapsf_corr_ext',
 'sigmagap',
 'sigmagap_corr',
 'ra',
 'dec',
 'rb',
 'rbversion',
 'drb',
 'magapbig',
 'sigmagapbig',
 'rfid',
 'has_stamp',
 'corrected',
 'dubious',
 'candid_alert',
 'step_id_corr',
 'phase',
 'parent_candid']

## Construct methods to extract stamps data<a class="anchor" id="methods"></a>

In [4]:
def getStampsData(oid):
    stamps = {}
    stamps['oid']=oid
    stamps['candids']=[]
    detections = client.query_detections(oid=oid, format='pandas')
    for candid_name in detections['candid']:
        candid_stamps = client.get_stamps(oid, candid=candid_name)
        if candid_stamps != None:
            stamps['candids'].append((candid_name, candid_stamps))
    return stamps

A note that the ```candid_stamps``` are an ```astropy.io.fits.hdu.hdulist.HDUList``` object.

In [5]:
def saveStampsData(oid):
    stamps = getStampsData(oid)
    parent_dir=oid
    os.mkdir(parent_dir)
    for t in stamps['candids']:
        candid_name, candid = t
        filename = '%s_%s.fits' % (oid, candid_name)
        path = os.path.join(parent_dir, filename)
        candid.writeto(path)
        print('Saved : %s' % path)

In [6]:
def plotStampsData(oid):
    detections = client.query_detections(oid=oid, format='pandas')
    total_candids=len(detections['candid'])
    plotted_candids=0
    
    for c in detections['candid']:
        candid_stamps = client.get_stamps(oid, candid=c)
        if candid_stamps != None:
            client.plot_stamps(oid, candid=c)
            plotted_candids += 1
    
    print("Available candids plotted: %i / %i" % (plotted_candids, total_candids))

We can call ```saveStampsData()``` to save each stamp (with science, reference, and differene images) as a ```.fits``` file in a separate folder.

In [8]:
saveStampsData(obj_oid)

Saved : ZTF19acymzwg/ZTF19acymzwg_1092553052115015022.fits
Saved : ZTF19acymzwg/ZTF19acymzwg_1092572082115015003.fits
Saved : ZTF19acymzwg/ZTF19acymzwg_1096516052115015012.fits
Saved : ZTF19acymzwg/ZTF19acymzwg_1099527442115015024.fits
Saved : ZTF19acymzwg/ZTF19acymzwg_1099554912115015025.fits
Saved : ZTF19acymzwg/ZTF19acymzwg_1102557632115015020.fits


and let's plot the stamps of our ZTF object, just to see how they look:

In [7]:
plotStampsData(obj_oid)

/Users/cxc/anaconda3/lib/python3.8/site-packages/alerce/stamps.py:136: RuntimeWarning: AVRO File not found.
  warnings.warn("AVRO File not found.", RuntimeWarning)


Available candids plotted: 6 / 8
